In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle
import tensorflow as tf

In [2]:
# Loading the dataset
dataset = pd.read_csv("Churn_Modelling.csv")
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
# Preprocessing dataset
# Removing unwanted columns or features
dataset.drop(["RowNumber", "CustomerId", "Surname"], axis=1, inplace=True)
dataset.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
# Encoding the Categorical values
gender_label = LabelEncoder()
dataset["Gender"] = gender_label.fit_transform(dataset["Gender"])
dataset.head()


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [5]:
# One Hot Encoding on Geography column
#dataset["Geography"] = OneHotEncoder().fit_transform(dataset["Geography"].values.reshape(-1, 1)).toarray()
onehot_encoder_geo=OneHotEncoder(sparse_output=False)
geo_encode= onehot_encoder_geo.fit_transform(dataset[['Geography']])
#dataset["Geography"] = onehot_encoder_geo.fit_transform(dataset[["Geography"]].values.reshape(-1, 1))
geo_encode

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [6]:
onehot_encoder_geo.get_feature_names_out()

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [7]:
geo_encoded_df= pd.DataFrame(geo_encode,columns=onehot_encoder_geo.get_feature_names_out())

In [8]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [9]:
## Combining all the columns with original dataset
dataset=pd.concat([dataset.drop('Geography',axis=1),geo_encoded_df],axis=1)
dataset.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [10]:
# Divide the dataset into dependent and independent features
X= dataset.drop('Exited',axis=1)
y= dataset['Exited']

# Splitting the dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#scaling these features
scaler= StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)


In [11]:
#Save the encoders and scalers
with open ('label_encoder_gender.pkl','wb') as file:
    pickle.dump(gender_label,file)

with open ('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo,file)

with open ('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

In [12]:
X_train.shape

(8000, 12)

****ANN Implementation****

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense 
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime 

In [14]:
#Building ANN Model
model=Sequential([
    Dense(64,activation='relu', input_shape=(X_train.shape[1],)), ###First HL connected to input layer
    Dense(32,activation='relu'), ###HL2
    Dense(1,activation='sigmoid') ####Output layer
]
)

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
optimize=tf.keras.optimizers.Adam(learning_rate=0.01)
los=tf.keras.losses.BinaryCrossentropy()

In [17]:
##Compiling the model
model.compile(optimizer=optimize,loss=los,metrics=['accuracy'])

In [18]:
##Setting the Tensorboard
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping

log_dir = "logs/fit" +datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callbacks = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [19]:
## Setup the Early Stopping
early_stopping_callback= EarlyStopping(monitor='val_loss', patience=20, restore_best_weights= True)

In [20]:
## Training the Model
history=model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorflow_callbacks,early_stopping_callback]
)

Epoch 1/100


250/250 [==============================] - 15s 24ms/step - loss: 0.4052 - accuracy: 0.8299 - val_loss: 0.3550 - val_accuracy: 0.8530
Epoch 2/100
250/250 [==============================] - 2s 7ms/step - loss: 0.3547 - accuracy: 0.8531 - val_loss: 0.3460 - val_accuracy: 0.8575
Epoch 3/100
250/250 [==============================] - 2s 7ms/step - loss: 0.3506 - accuracy: 0.8568 - val_loss: 0.3455 - val_accuracy: 0.8605
Epoch 4/100
250/250 [==============================] - 2s 8ms/step - loss: 0.3448 - accuracy: 0.8608 - val_loss: 0.3593 - val_accuracy: 0.8540
Epoch 5/100
250/250 [==============================] - 2s 7ms/step - loss: 0.3405 - accuracy: 0.8596 - val_loss: 0.3455 - val_accuracy: 0.8570
Epoch 6/100
250/250 [==============================] - 2s 6ms/step - loss: 0.3372 - accuracy: 0.8625 - val_loss: 0.3488 - val_accuracy: 0.8510
Epoch 7/100
250/250 [==============================] - 1s 6ms/step - loss: 0.3365 - accuracy: 0.8634 - val_loss: 0.3446 - val_accuracy: 0.

In [21]:
model.save('model.h5')

e:\Courses\Udemy\Practice\NLP_Practice\New_Annclassification\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [22]:
# Load TensorBoard Extension
%load_ext tensorboard

In [24]:
%tensorboard --logdir logs/fit20250310-150610/

In [ ]:
%reload_ext tensorboard